In [1]:
%tensorflow_version 2.0.0

`%tensorflow_version` only switches the major version: `1.x` or `2.x`.
You set: `2.0.0`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.


In [2]:
import logging
from os.path import join as path_join

from google.colab import drive
drive.mount('/content/gdrive')
data_dir = 'gdrive/My Drive/kaggle/google-quest-challenge'
code_dir = 'gdrive/My Drive/kaggle/google-quest-code/post_comp_code'
logging.basicConfig(filename=path_join('gdrive','My Drive','kaggle','google-quest-code','post_comp_code','bertusecrawl_seed1977.log') ,level=logging.WARNING)
usefeatures_path = path_join(data_dir,'4/') #path_join(data_dir,'use/') 

#data_dir = '/home/jupyter/google_quest_datasets/'
#logging.basicConfig(filename=path_join('/home','jupyter','submissions','bert_use_crawl_seed_1977','bertusecrawl_seed1977.log') ,level=logging.WARNING)
#usefeatures_path = path_join(data_dir,'use/')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import subprocess
import sys

reqs = subprocess.check_output([sys.executable, '-m', 'pip', 'freeze'])
installed_packages = [r.decode().split('==')[0] for r in reqs.split()]

if 'sacremoses' not in installed_packages:
   !pip install sacremoses >/dev/null

if 'transformers' not in installed_packages:
   !pip install transformers >/dev/null
else:
   import transformers
   print(transformers.__version__)


In [4]:
import os
import transformers, sys, os, gc
import numpy as np, pandas as pd, math
import torch, random, os, multiprocessing, glob
import torch.nn.functional as F
import torch, time

#sys.path.insert(0, "gdrive/My Drive/kaggle/google-quest-challenge/")
sys.path.insert(0, data_dir)

from ml_stratifiers import MultilabelStratifiedShuffleSplit, MultilabelStratifiedKFold
from scipy.stats import spearmanr
from torch import nn
from torch.utils import data
from torch.utils.data import DataLoader, Dataset
from transformers import (
    BertTokenizer, BertModel, BertForSequenceClassification, BertConfig,
    WEIGHTS_NAME, CONFIG_NAME, AdamW, get_linear_schedule_with_warmup, 
    get_cosine_schedule_with_warmup, BertPreTrainedModel,
)
from os.path import join as path_join
import html

from tqdm import tqdm

#wiki/crawl
import re
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import pickle

print(transformers.__version__)

def set_seeds(SEED=42):
  os.environ["PYTHONHASHSEED"] = str(SEED)
  np.random.seed(SEED)
  torch.manual_seed(SEED)
  torch.cuda.manual_seed(SEED)
  torch.backends.cudnn.deterministic = True

SEED=2019
set_seeds(SEED)
HEAD_TAIL = True

train_csv_path = path_join(data_dir,'train.csv')
test_csv_path = path_join(data_dir,'test.csv')

device = 'cuda' if torch.cuda.is_available() else 'cpu'


2.4.1


Using TensorFlow backend.


# Get crawl embedding matrix

In [0]:
# added preprocessing from https://www.kaggle.com/wowfattie/3rd-place/data

def sigmoid(x):
    return 1 / (1 + np.exp(-x))


puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£',
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', '\n', '\xa0', '\t',
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', '\u3000', '\u202f',
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', '«',
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]
mispell_dict = {"aren't" : "are not", "aren  '  t" : "are not",
"can't" : "cannot", "can  '  t" : "cannot",
"couldn't" : "could not","couldn  '  t" : "could not",
"couldnt" : "could not",
"didn't" : "did not","didn  '  t" : "did not",
"doesn't" : "does not","doesn  '  t" : "does not",
"doesnt" : "does not", 
"don't" : "do not","don  '  t" : "do not",
"hadn't" : "had not","hadn  '  t" : "had not",
"hasn't" : "has not", "hasn  '  t" : "has not",
"haven't" : "have not", "haven  '  t" : "have not",
"havent" : "have not",  
"he'd" : "he would", "he  '  d" : "he would",
"he'll" : "he will","he  '  ll" : "he will",
"he's" : "he is","he  '  s" : "he is",
"i'd" : "I would","i  '  d" : "I would",
"i'd" : "I had","i  '  d" : "I had",
"i'll" : "I will",  "i  '  ll" : "I will",
"i'm" : "I am",  "i  '  m" : "I am",
"isn't" : "is not", "isn  '  t" : "is not",
"it's" : "it is",  "it  '  s" : "it is",
"it'll":"it will",   "it  '  ll":"it will",
"i've" : "I have",   "i  '  ve" : "I have",
"let's" : "let us",  "let  '  s" : "let us",
"mightn't" : "might not",  "mightn  '  t" : "might not",
"mustn't" : "must not","mustn  '  t" : "must not",
"shan't" : "shall not","shan  '  t" : "shall not",
"she'd" : "she would","she  '  d" : "she would",
"she'll" : "she will","she  '  ll" : "she will",
"she's" : "she is","she  '  s" : "she is",
"shouldn't" : "should not","shouldn  '  t" : "should not",
"shouldnt" : "should not",
"that's" : "that is","that  '  s" : "that is",
"thats" : "that is",
"there's" : "there is","there  '  s" : "there is",
"theres" : "there is",
"they'd" : "they would","they  '  d" : "they would",
"they'll" : "they will","they  '  ll" : "they will",
"they're" : "they are","they  '  re" : "they are",
"theyre":  "they are",
"they've" : "they have","they  '  ve" : "they have",
"we'd" : "we would","we  '  d" : "we would",
"we're" : "we are","we  '  re" : "we are",
"weren't" : "were not","weren  '  t" : "were not",
"we've" : "we have","we  '  ve" : "we have",
"what'll" : "what will","what  '  ll" : "what will",
"what're" : "what are","what  '  re" : "what are",
"what's" : "what is","what  '  s" : "what is",
"what've" : "what have","what  '  ve" : "what have",
"where's" : "where is","where  '  s" : "where is",
"who'd" : "who would","who  '  d" : "who would",
"who'll" : "who will","who  '  ll" : "who will",
"who're" : "who are","who  '  re" : "who are",
"who's" : "who is","who  '  s" : "who is",
"who've" : "who have","who  '  ve" : "who have",
"won't" : "will not","won  '  t" : "will not",
"wouldn't" : "would not","wouldn  '  t" : "would not",
"you'd" : "you would","you  '  d" : "you would",

"you'll" : "you will","you  '  ll" : "you will",
"you're" : "you are","you  '  re" : "you are",
"you've" : "you have","you  '  ve" : "you have",
"'re": " are",
"wasn't": "was not","wasn  '  t": "was not",
"we'll":" will","we  '  ll":" will",
"didn't": "did not","didn  '  t": "did not",
"tryin'":"trying"}


def clean_text(x):
    x = str(x)
    for punct in puncts:
        x = x.replace(punct, f' {punct} ')
    return x


def clean_numbers(x):
    x = re.sub('[0-9]{5,}', '#####', x)
    x = re.sub('[0-9]{4}', '####', x)
    x = re.sub('[0-9]{3}', '###', x)
    x = re.sub('[0-9]{2}', '##', x)
    return x


def _get_mispell(mispell_dict):
    mispell_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))
    return mispell_dict, mispell_re


def replace_typical_misspell(text):
    mispellings, mispellings_re = _get_mispell(mispell_dict)

    def replace(match):
        return mispellings[match.group(0)]

    return mispellings_re.sub(replace, text)


def clean_data(df, columns: list):
    for col in columns:
        df[col] = df[col].apply(lambda x: clean_numbers(x))
        df[col] = df[col].apply(lambda x: clean_text(x.lower()))
        df[col] = df[col].apply(lambda x: replace_typical_misspell(x))

    return df

In [0]:

def get_coefs(word,*arr):
    return word, np.asarray(arr, dtype='float32')


def load_embeddings(path):
    with open(path,'rb') as f:
        emb_arr = pickle.load(f)
    return emb_arr


def build_matrix(embedding_path: str = '',
                 embedding_path_spellcheck: str = path_join(data_dir, 'wiki-news-300d-1M.vec'),
                 word_dict: dict = None, max_features: int = 100000,
                 embed_size: int= 300, ):

    # embedding_index = dict(get_coefs(*o.strip().split(" ")) for o in open(embedding_path, encoding='utf-8'))
    embedding_index = load_embeddings(embedding_path)

    nb_words = min(max_features, len(word_dict))
    embedding_matrix = np.zeros((nb_words + 1, embed_size))
    unknown_words = []
    for word, i in word_dict.items():
        key = word
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
            continue
        embedding_vector = embedding_index.get(word.lower())
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
            continue
        embedding_vector = embedding_index.get(word.upper())
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
            continue
        embedding_vector = embedding_index.get(word.capitalize())
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
            continue
        unknown_words.append(key)

    print(f'{len(unknown_words) * 100 / len(word_dict):.4f}% words are not in embeddings')
    return embedding_matrix, nb_words, unknown_words


In [0]:
test = pd.read_csv(test_csv_path).fillna(' ')
train = pd.read_csv(train_csv_path).fillna(' ')

# train.question_body = train.question_body.apply(html.unescape)
# train.answer        = train.answer.apply(html.unescape)

In [0]:
train = clean_data(train, ['answer', 'question_body', 'question_title'])
test = clean_data(test, ['answer', 'question_body', 'question_title'])

In [0]:
tokenizer = Tokenizer()
full_text = list(train['question_body']) + \
                       list(train['answer']) + \
                       list(train['question_title']) + \
                       list(test['question_body']) + \
                       list(test['answer']) + \
                       list(test['question_title'])
tokenizer.fit_on_texts(full_text)

In [10]:
embed_size=300
embedding_path = path_join(data_dir,'crawl-300d-2M.pkl') #"/kaggle/input/pickled-crawl300d2m-for-kernel-competitions/crawl-300d-2M.pkl"

embedding_matrix, nb_words, unknown_words = build_matrix(embedding_path,  path_join(data_dir,'wiki-news-300d-1M.vec') , tokenizer.word_index,
                                              100000, embed_size)

25.7148% words are not in embeddings


In [0]:
#pickle dump embedding matrix 

# with open("embedding_matrix.pickle", "wb") as output_file:
#       pickle.dump(embedding_matrix, output_file)

# with open("embedding_matrix.pickle", "rb") as input_file:
#       embedding_matrix = pickle.load(input_file)
 

 #pickle dump embedding matrix 

with open(path_join(code_dir,"embedding_matrix.pickle"), "wb") as output_file:
      pickle.dump(embedding_matrix, output_file)

with open(path_join(code_dir,"embedding_matrix.pickle"), "rb") as input_file:
      embedding_matrix = pickle.load(input_file)
 

### Create USE Features 

In [0]:
import os
import re
import gc
import pickle  
import random
import keras

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
#import keras.backend as K

#from keras.models import Model
#from keras.layers import Dense, Input, Dropout, Lambda
#from keras.optimizers import Adam
#from keras.callbacks import Callback
from scipy.stats import spearmanr, rankdata
from os.path import join as path_join
from numpy.random import seed
from urllib.parse import urlparse
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold
from sklearn.linear_model import MultiTaskElasticNet
import re

seed(SEED)
#tf.random.set_seed(SEED)
random.seed(SEED)

In [13]:
train = pd.read_csv(path_join(data_dir, 'train.csv'))
test = pd.read_csv(path_join(data_dir, 'test.csv'))
# train.question_body = train.question_body.apply(html.unescape)
# train.answer        = train.answer.apply(html.unescape)

print(train.shape, test.shape)

(6079, 41) (476, 11)


## Features 

In [0]:
tmp = pd.DataFrame([['unknown1', 'LIFE_ARTS'],['unknown2', 'CULTURE'],['unknown3', 'SCIENCE'],
             ['unknown4', 'STACKOVERFLOW'],['unknown5', 'TECHNOLOGY'],['unknown6', 'LIFE_ARTS'],
              ['unknown7', 'CULTURE'],['unknown8', 'SCIENCE'],['unknown9', 'STACKOVERFLOW'],
              ['unknown10', 'TECHNOLOGY'],['unknown11', 'CULTURE'],['unknown12', 'SCIENCE'],
              ['unknown13', 'STACKOVERFLOW'],['unknown14', 'TECHNOLOGY'],['unknown15', 'LIFE_ARTS'],
              ['unknown16', 'LIFE_ARTS'],['unknown17', 'CULTURE'],['unknown18', 'SCIENCE'],
             ['unknown19', 'STACKOVERFLOW'],['unknown20', 'TECHNOLOGY'],['unknown21', 'LIFE_ARTS'],
              ['unknown22', 'CULTURE'],['unknown23', 'SCIENCE'],['unknown24', 'STACKOVERFLOW'],
              ['unknown25', 'TECHNOLOGY'],['unknown26', 'CULTURE'],['unknown27', 'SCIENCE'],
              ['unknown28', 'STACKOVERFLOW'],['unknown29', 'TECHNOLOGY'],['unknown30', 'LIFE_ARTS'] ,
                    ['unknown31', 'LIFE_ARTS'],['unknown32', 'CULTURE'],['unknown33', 'SCIENCE'],
             ['unknown34', 'STACKOVERFLOW'],['unknown35', 'TECHNOLOGY'],['unknown36', 'LIFE_ARTS'],
              ['unknown37', 'CULTURE'],['unknown38', 'SCIENCE'],['unknown39', 'STACKOVERFLOW'],
              ['unknown40', 'TECHNOLOGY']
             ],columns =  ['netloc', 'category'])

In [15]:
find = re.compile(r"^[^.]*")

train['netloc'] = train['url'].apply(lambda x: re.findall(find, urlparse(x).netloc)[0])
test['netloc'] = test['url'].apply(lambda x: re.findall(find, urlparse(x).netloc)[0])

features = ['netloc', 'category']
merged = pd.concat([train[features], test[features]])

#Adding unknown url
merged = merged.append(tmp)

ohe = OneHotEncoder()
ohe.fit(merged)

features_train = ohe.transform(train[features]).toarray()
features_test = ohe.transform(test[features]).toarray()
print(features_train.shape, features_test.shape)
#Need to add 40 more unknown categories for private test set !! 

(6079, 104) (476, 104)


In [0]:
module_url = usefeatures_path
embed = hub.load(module_url)

In [0]:
input_columns = ['question_title', 'question_body', 'answer']

In [18]:
embeddings_train = {}
embeddings_test = {}
for text in input_columns:
    print(text)
    train_text = train[text].str.replace('?', '.').str.replace('!', '.').tolist()
    test_text = test[text].str.replace('?', '.').str.replace('!', '.').tolist()
    
    curr_train_emb = []
    curr_test_emb = []
    batch_size = 4
    ind = 0
    while ind*batch_size < len(train_text):
        curr_train_emb.append(embed(train_text[ind*batch_size: (ind + 1)*batch_size])["outputs"].numpy())
        ind += 1
        
    ind = 0
    while ind*batch_size < len(test_text):
        curr_test_emb.append(embed(test_text[ind*batch_size: (ind + 1)*batch_size])["outputs"].numpy())
        ind += 1    
        
    embeddings_train[text + '_embedding'] = np.vstack(curr_train_emb)
    embeddings_test[text + '_embedding'] = np.vstack(curr_test_emb)
    
del embed
gc.collect()

question_title
question_body
answer


44

In [0]:
#pickle dump embeddings_train #use features

with open(path_join(code_dir,"embeddings_train.pickle"), "wb") as output_file:
      pickle.dump(embeddings_train, output_file)

with open(path_join(code_dir,"embeddings_train.pickle"), "rb") as input_file:
      embeddings_train = pickle.load(input_file)
 

### Distance features

In [0]:
l2_dist = lambda x, y: np.power(x - y, 2).sum(axis=1)

cos_dist = lambda x, y: (x*y).sum(axis=1)

dist_features_train = np.array([
   l2_dist(embeddings_train['question_title_embedding'], embeddings_train['answer_embedding']),
   l2_dist(embeddings_train['question_body_embedding'], embeddings_train['answer_embedding']),
   l2_dist(embeddings_train['question_body_embedding'], embeddings_train['question_title_embedding']),
   cos_dist(embeddings_train['question_title_embedding'], embeddings_train['answer_embedding']),
   cos_dist(embeddings_train['question_body_embedding'], embeddings_train['answer_embedding']),
   cos_dist(embeddings_train['question_body_embedding'], embeddings_train['question_title_embedding'])
]).T

dist_features_test = np.array([
    l2_dist(embeddings_test['question_title_embedding'], embeddings_test['answer_embedding']),
    l2_dist(embeddings_test['question_body_embedding'], embeddings_test['answer_embedding']),
    l2_dist(embeddings_test['question_body_embedding'], embeddings_test['question_title_embedding']),
    cos_dist(embeddings_test['question_title_embedding'], embeddings_test['answer_embedding']),
    cos_dist(embeddings_test['question_body_embedding'], embeddings_test['answer_embedding']),
    cos_dist(embeddings_test['question_body_embedding'], embeddings_test['question_title_embedding'])
]).T

X_train = np.hstack([item for k, item in embeddings_train.items()] + [features_train, dist_features_train])
X_test = np.hstack([item for k, item in embeddings_test.items()] + [features_test, dist_features_test])

In [0]:
#pickle dump embeddings_train #use features

with open(path_join(code_dir,"X_train.pickle"), "wb") as output_file:
      pickle.dump(X_train, output_file)

with open(path_join(code_dir,"X_train.pickle"), "rb") as input_file:
      X_train = pickle.load(input_file)
 

In [0]:
# unique_hosts = list(set(train['host'].unique().tolist() + test['host'].unique().tolist()))
# host_dict = {i + 1: e for i, e in enumerate(unique_hosts)}
# host_dict_reverse = {v: k for k, v in host_dict.items()}

# unique_categories = list(set(train['category'].unique().tolist() + test['category'].unique().tolist()))
# category_dict = {i + 1: e for i, e in enumerate(unique_categories)}
# category_dict_reverse = {v: k for k, v in category_dict.items()}
max_len = 500
max_len_title = 30
train_question_tokenized = pad_sequences(tokenizer.texts_to_sequences(train['question_body']), maxlen = max_len)
train_answer_tokenized = pad_sequences(tokenizer.texts_to_sequences(train['answer']), maxlen = max_len)
train_title_tokenized = pad_sequences(tokenizer.texts_to_sequences(train['question_title']), maxlen = max_len_title)

### Model

In [0]:
class SpatialDropout(nn.Dropout2d):
    def forward(self, x):
        x = x.unsqueeze(2)    # (N, T, 1, K)
        x = x.permute(0, 3, 2, 1)  # (N, K, 1, T)
        x = super(SpatialDropout, self).forward(x)  # (N, K, 1, T), some features are masked
        x = x.permute(0, 3, 2, 1)  # (N, T, 1, K)
        x = x.squeeze(2)  # (N, T, K)
        return x


In [0]:
#Change the model to include only question+title embedding and answer+title embedding
LEN_USE_FEATURES = 1646 #1606 
class BertForSequenceClassification_v2Answer(BertPreTrainedModel):
    r"""
        **labels**: (`optional`) ``torch.LongTensor`` of shape ``(batch_size,)``:
            Labels for computing the sequence classification/regression loss.
            Indices should be in ``[0, ..., config.num_labels - 1]``.
            If ``config.num_labels == 1`` a regression loss is computed (Mean-Square loss),
            If ``config.num_labels > 1`` a classification loss is computed (Cross-Entropy).

    Outputs: `Tuple` comprising various elements depending on the configuration (config) and inputs:
        **loss**: (`optional`, returned when ``labels`` is provided) ``torch.FloatTensor`` of shape ``(1,)``:
            Classification (or regression if config.num_labels==1) loss.
        **logits**: ``torch.FloatTensor`` of shape ``(batch_size, config.num_labels)``
            Classification (or regression if config.num_labels==1) scores (before SoftMax).
        **hidden_states**: (`optional`, returned when ``config.output_hidden_states=True``)
            list of ``torch.FloatTensor`` (one for the output of each layer + the output of the embeddings)
            of shape ``(batch_size, sequence_length, hidden_size)``:
            Hidden-states of the model at the output of each layer plus the initial embedding outputs.
        **attentions**: (`optional`, returned when ``config.output_attentions=True``)
            list of ``torch.FloatTensor`` (one for each layer) of shape ``(batch_size, num_heads, sequence_length, sequence_length)``:
            Attentions weights after the attention softmax, used to compute the weighted average in the self-attention heads.

    Examples::

        tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
        input_ids = torch.tensor(tokenizer.encode("Hello, my dog is cute", add_special_tokens=True)).unsqueeze(0)  # Batch size 1
        labels = torch.tensor([1]).unsqueeze(0)  # Batch size 1
        outputs = model(input_ids, labels=labels)
        loss, logits = outputs[:2]

    """

    def __init__(self, config,embedding_matrix=None):
        
        #super(BertForSequenceClassification, self).__init__(config)
        super(BertForSequenceClassification_v2Answer,self).__init__(config)
        self.num_labels = config.num_labels

        self.embedding = nn.Embedding(*embedding_matrix.shape) 
        self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding.weight.requires_grad = False
        self.embedding_dropout = SpatialDropout(0.3)

        self.linear_q_add = nn.Linear(300, 128)
        self.linear_q_add1 = nn.Linear(128, self.config.num_labels)
       
        # MINE - works
        self.bert = BertModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.linear1 = nn.Linear(config.hidden_size+LEN_USE_FEATURES +  self.config.num_labels  ,config.hidden_size+LEN_USE_FEATURES +  self.config.num_labels)
        self.classifier = nn.Linear(config.hidden_size+LEN_USE_FEATURES +  self.config.num_labels  , self.config.num_labels)

        self.init_weights()

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        usefeatures = None, 
        question_data = None,
        answer_data = None,
        title_data = None
    ):

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
        )
       
        last_hidden_state = outputs[0][:,0,:].reshape((-1, 1, 768)) #CLS Token 
        pooled_output = outputs[1] 
        hidden_states = outputs[2]

        # MINE - works
        h12 = hidden_states[-1][:, 0].reshape((-1, 1, 768))
        h11 = hidden_states[-2][:, 0].reshape((-1, 1, 768))
        h10 = hidden_states[-3][:, 0].reshape((-1, 1, 768))
        h9  = hidden_states[-4][:, 0].reshape((-1, 1, 768))
        all_h = torch.cat([ h9, h10, h11, h12], 1)
        all_h_mean = torch.mean(all_h, 1).reshape((-1,1,768))
        mean_pool = torch.mean(torch.cat([last_hidden_state,all_h_mean],1),1)
        pooled_output = self.dropout(mean_pool)


        #Embedding 
        h_embedding_q = self.embedding(question_data)
        h_embedding_q = self.embedding_dropout(h_embedding_q)

        h_embedding_a = self.embedding(answer_data)
        h_embedding_a = self.embedding_dropout(h_embedding_a)

        h_embedding_t = self.embedding(title_data)
        h_embedding_t = self.embedding_dropout(h_embedding_t)

        add = torch.cat((h_embedding_t, h_embedding_q, h_embedding_a), 1) #h_embedding_a,
        add = self.linear_q_add(torch.mean(add, 1))
        add = self.linear_q_add1(add)

        x = torch.nn.ELU()(self.linear1(torch.cat([pooled_output,usefeatures,add],1)))
        logits = self.classifier(x)

        #logits = self.classifier(pooled_output)
        outputs = (logits,) + outputs[2:]  # add hidden states and attention if they are here

        if labels is not None:
            if self.num_labels == 1:
                #  We are doing regression
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1), labels.view(-1))
            else:
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            outputs = (loss,) + outputs

        return outputs  # (loss), logits, (hidden_states), (attentions)


In [0]:
#Change the model to include only question+title embedding and answer+title embedding
LEN_USE_FEATURES = 1646 #1606 
class BertForSequenceClassification_v2Question(BertPreTrainedModel):
    r"""
        **labels**: (`optional`) ``torch.LongTensor`` of shape ``(batch_size,)``:
            Labels for computing the sequence classification/regression loss.
            Indices should be in ``[0, ..., config.num_labels - 1]``.
            If ``config.num_labels == 1`` a regression loss is computed (Mean-Square loss),
            If ``config.num_labels > 1`` a classification loss is computed (Cross-Entropy).

    Outputs: `Tuple` comprising various elements depending on the configuration (config) and inputs:
        **loss**: (`optional`, returned when ``labels`` is provided) ``torch.FloatTensor`` of shape ``(1,)``:
            Classification (or regression if config.num_labels==1) loss.
        **logits**: ``torch.FloatTensor`` of shape ``(batch_size, config.num_labels)``
            Classification (or regression if config.num_labels==1) scores (before SoftMax).
        **hidden_states**: (`optional`, returned when ``config.output_hidden_states=True``)
            list of ``torch.FloatTensor`` (one for the output of each layer + the output of the embeddings)
            of shape ``(batch_size, sequence_length, hidden_size)``:
            Hidden-states of the model at the output of each layer plus the initial embedding outputs.
        **attentions**: (`optional`, returned when ``config.output_attentions=True``)
            list of ``torch.FloatTensor`` (one for each layer) of shape ``(batch_size, num_heads, sequence_length, sequence_length)``:
            Attentions weights after the attention softmax, used to compute the weighted average in the self-attention heads.

    Examples::

        tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
        input_ids = torch.tensor(tokenizer.encode("Hello, my dog is cute", add_special_tokens=True)).unsqueeze(0)  # Batch size 1
        labels = torch.tensor([1]).unsqueeze(0)  # Batch size 1
        outputs = model(input_ids, labels=labels)
        loss, logits = outputs[:2]

    """

    def __init__(self, config,embedding_matrix=None):
        
        #super(BertForSequenceClassification, self).__init__(config)
        super(BertForSequenceClassification_v2Question,self).__init__(config)
        self.num_labels = config.num_labels

        self.embedding = nn.Embedding(*embedding_matrix.shape) 
        self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding.weight.requires_grad = False
        self.embedding_dropout = SpatialDropout(0.3)

        self.linear_q_add = nn.Linear(300, 128)
        
        self.linear_q_add1 = nn.Linear(128, self.config.num_labels)

        
        # MINE - works
        self.bert = BertModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.linear1 = nn.Linear(config.hidden_size+LEN_USE_FEATURES +  self.config.num_labels  ,config.hidden_size+LEN_USE_FEATURES +  self.config.num_labels)
        self.classifier = nn.Linear(config.hidden_size+LEN_USE_FEATURES +  self.config.num_labels  , self.config.num_labels)

        self.init_weights()

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        usefeatures = None, 
        question_data = None,
        answer_data = None,
        title_data = None
    ):

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
        )

        last_hidden_state = outputs[0][:,0,:].reshape((-1, 1, 768)) #CLS Token 
        pooled_output = outputs[1] 
        hidden_states = outputs[2]

        # MINE - works
        h12 = hidden_states[-1][:, 0].reshape((-1, 1, 768))
        h11 = hidden_states[-2][:, 0].reshape((-1, 1, 768))
        h10 = hidden_states[-3][:, 0].reshape((-1, 1, 768))
        h9  = hidden_states[-4][:, 0].reshape((-1, 1, 768))
        all_h = torch.cat([ h9, h10, h11, h12], 1)
        all_h_mean = torch.mean(all_h, 1).reshape((-1,1,768))
        mean_pool = torch.mean(torch.cat([last_hidden_state,all_h_mean],1),1)
        pooled_output = self.dropout(mean_pool)


        #Embedding 
        h_embedding_q = self.embedding(question_data)
        h_embedding_q = self.embedding_dropout(h_embedding_q)

        h_embedding_t = self.embedding(title_data)
        h_embedding_t = self.embedding_dropout(h_embedding_t)

        add = torch.cat((h_embedding_t,  h_embedding_q), 1) #h_embedding_a,
        add = self.linear_q_add(torch.mean(add,1))
        add = self.linear_q_add1(add)
        
        x = torch.nn.ELU()(self.linear1(torch.cat([pooled_output,usefeatures,add],1)))
        logits = self.classifier(x)

        
        #logits = self.classifier(pooled_output)
        outputs = (logits,) + outputs[2:]  # add hidden states and attention if they are here

        if labels is not None:
            if self.num_labels == 1:
                #  We are doing regression
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1), labels.view(-1))
            else:
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            outputs = (loss,) + outputs

        return outputs  # (loss), logits, (hidden_states), (attentions)


In [0]:

train = pd.read_csv( path_join(data_dir,"train.csv"))
test = pd.read_csv( path_join(data_dir, "test.csv"))
# train.question_body = train.question_body.apply(html.unescape)
# train.answer        = train.answer.apply(html.unescape)

target_cols_questions = ['question_asker_intent_understanding', 'question_body_critical', 
               'question_conversational', 'question_expect_short_answer', 
               'question_fact_seeking', 'question_has_commonly_accepted_answer', 
               'question_interestingness_others', 'question_interestingness_self', 
               'question_multi_intent', 'question_not_really_a_question', 
               'question_opinion_seeking', 'question_type_choice',
               'question_type_compare', 'question_type_consequence',
               'question_type_definition', 'question_type_entity', 
               'question_type_instructions', 'question_type_procedure', 
               'question_type_reason_explanation', 'question_type_spelling', 
               'question_well_written'] 

target_cols_answers =  ['answer_helpful',
               'answer_level_of_information', 'answer_plausible', 
               'answer_relevance', 'answer_satisfaction', 
               'answer_type_instructions', 'answer_type_procedure', 
               'answer_type_reason_explanation', 'answer_well_written']

In [0]:
weights_questions = np.ones((len(train),))
mask = np.sum(train[target_cols_questions]>0.5,1)
weights_questions = weights_questions + mask 
loss_weight_questions = 1.0 / weights_questions.mean()

train['weights_questions'] = weights_questions
target_cols_questions = target_cols_questions + ['weights_questions']


weights_answers = np.ones((len(train),))
mask = np.sum(train[target_cols_answers]>0.5,1)
weights_answers = weights_answers + mask 
loss_weight_answers = 1.0 / weights_answers.mean()

train['weights_answers'] = weights_answers
target_cols_answers = target_cols_answers + ['weights_answers']


In [0]:
# helpers
# From the Ref Kernel's
from math import floor, ceil

def _get_masks(tokens, max_seq_length):
    """Mask for padding"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))

def _get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    
    if len(tokens) > max_seq_length:
        raise IndexError("Token length more than max seq length!")
        
    segments = []
    first_sep = True
    current_segment_id = 0
    
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            if first_sep:
                first_sep = False 
            else:
                current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))

def _get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids

def compute_output_arrays(df, columns):
    return np.asarray(df[columns])

### Tokenize Answers

In [0]:


def _trim_input_answers(title, question, answer, tokenizer, max_sequence_length=512, t_max_len=30, q_max_len=239, a_max_len=239):
    
    #293+239+30 = 508 + 4 = 512
    
    t = tokenizer.tokenize(title)
    q = tokenizer.tokenize(question)
    a = tokenizer.tokenize(answer)
    
    t_len = len(t)
    q_len = len(q)
    a_len = len(a)

    if (t_len+q_len+a_len+4) > max_sequence_length:
        
        if t_max_len > t_len:
            t_new_len = t_len
            a_max_len = a_max_len + floor((t_max_len - t_len)/2)
            q_max_len = q_max_len + ceil((t_max_len - t_len)/2)
        else:
            t_new_len = t_max_len
      
        if a_max_len > a_len:
            a_new_len = a_len 
            q_new_len = q_max_len + (a_max_len - a_len)
        elif q_max_len > q_len:
            a_new_len = a_max_len + (q_max_len - q_len)
            q_new_len = q_len
        else:
            a_new_len = a_max_len
            q_new_len = q_max_len
            
            
        # if t_new_len+a_new_len+q_new_len+4 != max_sequence_length:
        #     raise ValueError("New sequence length should be %d, but is %d"%(max_sequence_length, (t_new_len + a_new_len + q_new_len + 4)))
        
        # t = t[:t_new_len]
        # q = q[:q_new_len]
        # a = a[:a_new_len]


        if t_new_len+a_new_len+q_new_len+4 != max_sequence_length:
            raise ValueError("New sequence length should be %d, but is %d"%(max_sequence_length, (t_new_len + a_new_len + q_new_len + 4)))
        
        q_len_head = round(q_new_len/2)
        q_len_tail = -1* (q_new_len -q_len_head)
        a_len_head = round(a_new_len/2)
        a_len_tail = -1* (a_new_len -a_len_head)        ## Head+Tail method .
        t = t[:t_new_len]
        if HEAD_TAIL :
            q = q[:q_len_head]+q[q_len_tail:]
            a = a[:a_len_head]+a[a_len_tail:]
        else:
            q = q[:q_new_len]
            a = a[:a_new_len] ## No Head+Tail ,usual processing
    
    return t, q, a



def _convert_to_bert_inputs_answers(title, question, answer, tokenizer, max_sequence_length):
    """Converts tokenized input to ids, masks and segments for BERT"""
    
    stoken = ["[CLS]"] + title + ["[SEP]"] + question + ["[SEP]"] + answer + ["[SEP]"] 

    input_ids = _get_ids(stoken, tokenizer, max_sequence_length)
    input_masks = _get_masks(stoken, max_sequence_length)
    input_segments = _get_segments(stoken, max_sequence_length)

    return [input_ids, input_masks, input_segments]



def compute_input_arays_answers(df, columns, tokenizer, max_sequence_length):
    
    input_ids, input_masks, input_segments = [], [], []
    for _, instance in tqdm(df[columns].iterrows(),position=0, leave=True ):
        t, q, a  = instance.question_title,instance.question_body, instance.answer
        t, q, a  = _trim_input_answers(t, q, a, tokenizer, max_sequence_length)
        ids, masks, segments = _convert_to_bert_inputs_answers(t,q, a,tokenizer, max_sequence_length)
        input_ids.append(ids)
        input_masks.append(masks)
        input_segments.append(segments)
    return [
        torch.from_numpy(np.asarray(input_ids, dtype=np.int32)).long(), 
        torch.from_numpy(np.asarray(input_masks, dtype=np.int32)).long(),
        torch.from_numpy(np.asarray(input_segments, dtype=np.int32)).long(),
    ]

### Tokenize Questions

In [0]:
def _trim_input(title, question, tokenizer, max_sequence_length=512, t_max_len=30, q_max_len=479):
    
    #3 + 479 + 30
    
    t = tokenizer.tokenize(title)
    q = tokenizer.tokenize(question)
    
    t_len = len(t)
    q_len = len(q)
    
    if (t_len+q_len+3) > max_sequence_length:
        
        if t_max_len > t_len:
            t_new_len = t_len
            q_max_len = q_max_len + (t_max_len - t_len)
            q_new_len = q_max_len
        else:
            t_new_len = t_max_len
            q_new_len = q_max_len
      
        if q_max_len > q_len:
            q_new_len = q_len
            
            
        if t_new_len+q_new_len+3 != max_sequence_length:
            raise ValueError("New sequence length should be %d, but is %d"%(max_sequence_length, (t_new_len + q_new_len + 3)))
        
        t = t[:t_new_len]
        q = q[:q_new_len]
        
    return t, q

def _convert_to_bert_inputs(title, question, tokenizer, max_sequence_length):
    """Converts tokenized input to ids, masks and segments for BERT"""
    
    stoken = ["[CLS]"] + title + ["[SEP]"] + question + ["[SEP]"] 

    input_ids = _get_ids(stoken, tokenizer, max_sequence_length)
    input_masks = _get_masks(stoken, max_sequence_length)
    input_segments = _get_segments(stoken, max_sequence_length)

    return [input_ids, input_masks, input_segments]

def compute_input_arays(df, columns, tokenizer, max_sequence_length):
    
    input_ids, input_masks, input_segments = [], [], []
    for _, instance in tqdm(df[columns].iterrows(),position=0, leave=True ):
        t, q  = instance.question_title, instance.question_body
        t, q  = _trim_input(t, q, tokenizer, max_sequence_length)
        ids, masks, segments = _convert_to_bert_inputs(t, q,tokenizer, max_sequence_length)
        input_ids.append(ids)
        input_masks.append(masks)
        input_segments.append(segments)
    return [
        torch.from_numpy(np.asarray(input_ids, dtype=np.int32)).long(), 
        torch.from_numpy(np.asarray(input_masks, dtype=np.int32)).long(),
        torch.from_numpy(np.asarray(input_segments, dtype=np.int32)).long(),
    ]




In [0]:
class QuestDataset(torch.utils.data.Dataset):
    def __init__(self, inputs, usefeatures,question_data, answer_data, title_data, lengths, labels = None):
        
        self.inputs = inputs
        self.usefeatures = usefeatures
        self.question_data = question_data
        self.answer_data = answer_data
        self.title_data = title_data
        if labels is not None:
            self.labels = labels
        else:
            self.labels = None
        self.lengths = lengths
       

    def __getitem__(self, idx):
        
        input_ids       = self.inputs[0][idx]
        input_masks     = self.inputs[1][idx]
        input_segments  = self.inputs[2][idx]
        lengths         = self.lengths[idx]
        usef            = self.usefeatures[idx,:]
        question_data   = self.question_data[idx,:]
        answer_data     = self.answer_data[idx,:]
        title_data      = self.title_data[idx,:]

        if self.labels is not None: # targets
            labels = self.labels[idx]
            return input_ids, input_masks, input_segments,  usef,  question_data, answer_data, title_data, labels, lengths
        return input_ids, input_masks, input_segments, usef,  question_data,answer_data, title_data,  lengths

    def __len__(self):
        return len(self.inputs[0])


In [0]:
def get_loss_fn_c_answer(criterion, lgits, lbls):
    return (criterion(lgits,lbls[:,0:9]) * lbls[:,9].view(-1,1)).mean()

def get_loss_fn_c(criterion, lgits, lbls):
    return (criterion(lgits,lbls[:,0:21]) * lbls[:,21].view(-1,1)).mean()



In [0]:
def train_model_answers(train_loader, optimizer, criterion, scheduler):
    
    model_a.train()
    avg_loss = 0.
    tk0 = tqdm(enumerate(train_loader),position=0, leave=True )
    
    for idx, batch in tk0:
        
        input_ids, input_masks, input_segments, usefeatures, question_data, answer_data, title_data,labels, _ = batch
        input_ids, input_masks, input_segments, usefeatures, question_data, answer_data, title_data,labels = input_ids.to(device), input_masks.to(device), input_segments.to(device), usefeatures.to(device) ,question_data.to(device),answer_data.to(device),title_data.to(device),labels.to(device)            
         
        output_train = model_a(input_ids = input_ids.long(),
                             labels = None,
                             attention_mask = input_masks,
                             token_type_ids = input_segments,
                             usefeatures = usefeatures,
                             question_data = question_data,
                             answer_data = answer_data,
                             title_data = title_data
                            )
        logits = output_train[0] #output preds

        loss = get_loss_fn_c_answer(criterion,logits,labels)
        #loss = get_loss_fn_b(criterion,logits,labels)
        #loss = loss / accumulation_steps    #https://gist.github.com/thomwolf/ac7a7da6b1888c2eeac8ac8b9b05d3d3
        loss.backward()
        
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        
        avg_loss += loss.item() / len(train_loader)
        del input_ids, input_masks, input_segments, labels, usefeatures, question_data, answer_data, title_data

    torch.cuda.empty_cache()
    gc.collect()
    return avg_loss

In [0]:
def train_model_questions(train_loader, optimizer, criterion, scheduler):
    
    model_q.train()
    avg_loss = 0.
    tk0 = tqdm(enumerate(train_loader),position=0, leave=True )
    
    for idx, batch in tk0:
        
        input_ids, input_masks, input_segments, usefeatures, question_data, answer_data, title_data, labels, _ = batch
        input_ids, input_masks, input_segments, usefeatures, question_data, answer_data, title_data, labels = input_ids.to(device), input_masks.to(device), input_segments.to(device),usefeatures.to(device),question_data.to(device),answer_data.to(device),title_data.to(device), labels.to(device)            
        output_train = model_q(input_ids = input_ids.long(),
                             labels = None,
                             attention_mask = input_masks,
                             token_type_ids = input_segments,
                             usefeatures = usefeatures,
                             question_data = question_data,
                             answer_data = answer_data,
                             title_data = title_data
                            )
        logits = output_train[0] #output preds

        loss = get_loss_fn_c(criterion,logits,labels)
        loss.backward()
        
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        
        avg_loss += loss.item() / len(train_loader)
        del input_ids, input_masks, input_segments, labels, usefeatures,  question_data, answer_data, title_data

    torch.cuda.empty_cache()
    gc.collect()
    return avg_loss

In [0]:
def val_model_answers(val_loader, val_shape, batch_size=8):  
    
    avg_val_loss = 0.
    model_a.eval() # eval mode
    
    valid_preds = np.zeros((val_shape, 9))
    original = np.zeros((val_shape, 9))
    
    tk0 = tqdm(enumerate(val_loader),position=0, leave=True )
    with torch.no_grad():
        
        for idx, batch in tk0:
            input_ids, input_masks, input_segments,usefeatures,  question_data, answer_data, title_data, labels, _ = batch
            input_ids, input_masks, input_segments, usefeatures,  question_data, answer_data, title_data, labels = input_ids.to(device), input_masks.to(device), input_segments.to(device), usefeatures.to(device), question_data.to(device),answer_data.to(device),title_data.to(device),labels.to(device)            
            
            output_val = model_a(input_ids = input_ids.long(),
                             labels = None,
                             attention_mask = input_masks,
                             token_type_ids = input_segments,
                             usefeatures = usefeatures,
                             question_data = question_data,
                             answer_data = answer_data,
                             title_data = title_data
                            )
            logits = output_val[0] #output preds
            
            vloss = get_loss_fn_c_answer(criterion,logits,labels)
            avg_val_loss += vloss.item() / len(val_loader)


            valid_preds[idx*batch_size : (idx+1)*batch_size] = logits.detach().cpu().squeeze().numpy()
            #original[idx*batch_size : (idx+1)*batch_size]    = labels.detach().cpu().squeeze().numpy()
            original[idx*batch_size : (idx+1)*batch_size]    = labels[:,0:9].detach().cpu().squeeze().numpy()
        
        score = 0
        preds = torch.sigmoid(torch.tensor(valid_preds)).numpy()

        rho_val = np.mean([spearmanr(original[:, i], preds[:,i]).correlation for i in range(preds.shape[1])])
        print('\r val_spearman-rho: %s' % (str(round(rho_val, 5))), end = 100*' '+'\n')
        
        for i in range(9):
            print(i, spearmanr(original[:,i], preds[:,i]))
            score += np.nan_to_num(spearmanr(original[:, i], preds[:, i]).correlation)
    return avg_val_loss, score/9. , preds

In [0]:
def val_model_questions(val_loader, val_shape, batch_size=8):  
    
    avg_val_loss = 0.
    model_q.eval() # eval mode
    
    valid_preds = np.zeros((val_shape, 21))
    original = np.zeros((val_shape, 21))
    
    tk0 = tqdm(enumerate(val_loader),position=0, leave=True )
    with torch.no_grad():
        
        for idx, batch in tk0:
            input_ids, input_masks, input_segments, usefeatures, question_data, answer_data, title_data, labels, _ = batch
            input_ids, input_masks, input_segments, usefeatures, question_data, answer_data, title_data, labels = input_ids.to(device), input_masks.to(device), input_segments.to(device),usefeatures.to(device), question_data.to(device),answer_data.to(device),title_data.to(device),labels.to(device)            
            
            output_val = model_q(input_ids = input_ids.long(),
                             labels = None,
                             attention_mask = input_masks,
                             token_type_ids = input_segments,
                             usefeatures = usefeatures,
                             question_data = question_data,
                             answer_data = answer_data,
                             title_data = title_data
                            )
            logits = output_val[0] #output preds
            
            vloss = get_loss_fn_c(criterion,logits,labels)
          
            avg_val_loss += vloss.item() / len(val_loader)


            valid_preds[idx*batch_size : (idx+1)*batch_size] = logits.detach().cpu().squeeze().numpy()
            #original[idx*batch_size : (idx+1)*batch_size]    = labels.detach().cpu().squeeze().numpy()
            original[idx*batch_size : (idx+1)*batch_size]    = labels[:,0:21].detach().cpu().squeeze().numpy()
        
        score = 0
        preds = torch.sigmoid(torch.tensor(valid_preds)).numpy()
       
        rho_val = np.mean([spearmanr(original[:, i], preds[:,i]).correlation for i in range(preds.shape[1])])
        print('\r val_spearman-rho: %s' % (str(round(rho_val, 5))), end = 100*' '+'\n')
        
        for i in range(21):
            print(i, spearmanr(original[:,i], preds[:,i]))
            score += np.nan_to_num(spearmanr(original[:, i], preds[:, i]).correlation)
    return avg_val_loss, score/21, preds #return 21 questions columns



In [37]:

set_seeds(SEED)

question_tokenizer = BertTokenizer.from_pretrained(path_join(data_dir,'bert_en_uncased_L-12_H-768_A-12', 'assets','vocab.txt'))
input_categories_q = list(train.columns[[1,2]]); 

bert_model_config_question = path_join(data_dir,'bert_config.json')
bert_config_q = BertConfig.from_json_file(bert_model_config_question)
bert_config_q.output_hidden_states = True
bert_config_q.num_labels = 21

answer_tokenizer = BertTokenizer.from_pretrained(path_join(data_dir,'bert_en_uncased_L-12_H-768_A-12', 'assets','vocab.txt'))
input_categories_a = list(train.columns[[1,5,2]]); 

bert_model_config_answer = path_join(data_dir,'bert_config.json')
bert_config_a = BertConfig.from_json_file(bert_model_config_answer)
bert_config_a.output_hidden_states = True
bert_config_a.num_labels = 9

device = 'cuda' if torch.cuda.is_available() else 'cpu'

class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'


# In[8]:

NUM_FOLDS = 5  # change this
BATCH_SIZE = 8
epochs = 4     # change this
ACCUM_STEPS = 1
test_predictions = [] 
all_rhos = []

kf = MultilabelStratifiedKFold(n_splits = NUM_FOLDS, random_state = SEED)

y_train_q = train[target_cols_questions].values # dummy
y_train_a = train[target_cols_answers].values # dummy

print(bcolors.FAIL, f"For Every Fold, Train {epochs} Epochs", bcolors.ENDC)

set_seeds(SEED)




 For Every Fold, Train 4 Epochs 


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


In [0]:
xtrain_usefeatures = X_train

In [39]:
best_rhos = []

for fold, (train_index, val_index) in enumerate(kf.split(train.values, y_train_q)):
  if fold > 2:  
    print(bcolors.HEADER, "Current Fold:", fold, bcolors.ENDC)
    train_df, val_df = train.iloc[train_index], train.iloc[val_index]
    print("Train and Valid Shapes are", train_df.shape, val_df.shape)   
    print(bcolors.HEADER, "Preparing train datasets....", bcolors.ENDC)

    #questions
    inputs_train_q = compute_input_arays(train_df, input_categories_q, question_tokenizer, max_sequence_length=512)
    outputs_train_q = compute_output_arrays(train_df, columns = target_cols_questions)
    outputs_train_q = torch.tensor(outputs_train_q, dtype=torch.float32)
    lengths_train_q = np.argmax(inputs_train_q[0] == 0, axis=1)
    lengths_train_q[lengths_train_q == 0] = inputs_train_q[0].shape[1]

    #answers
    inputs_train_a = compute_input_arays_answers(train_df, input_categories_a, answer_tokenizer, max_sequence_length=512)
    outputs_train_a = compute_output_arrays(train_df, columns = target_cols_answers)
    outputs_train_a = torch.tensor(outputs_train_a, dtype=torch.float32)
    lengths_train_a = np.argmax(inputs_train_a[0] == 0, axis=1)
    lengths_train_a[lengths_train_a == 0] = inputs_train_a[0].shape[1]

    print(bcolors.HEADER, "Preparing Valid datasets....", bcolors.ENDC)
    #questions
    inputs_valid_q = compute_input_arays(val_df, input_categories_q, question_tokenizer, max_sequence_length=512)
    outputs_valid_q = compute_output_arrays(val_df, columns = target_cols_questions)
    outputs_valid_q = torch.tensor(outputs_valid_q, dtype=torch.float32)
    lengths_valid_q = np.argmax(inputs_valid_q[0] == 0, axis=1)
    lengths_valid_q[lengths_valid_q == 0] = inputs_valid_q[0].shape[1]

    #answers
    inputs_valid_a = compute_input_arays_answers(val_df, input_categories_a, answer_tokenizer, max_sequence_length=512)
    outputs_valid_a = compute_output_arrays(val_df, columns = target_cols_answers)
    outputs_valid_a = torch.tensor(outputs_valid_a, dtype=torch.float32)
    lengths_valid_a = np.argmax(inputs_valid_a[0] == 0, axis=1)
    lengths_valid_a[lengths_valid_a == 0] = inputs_valid_a[0].shape[1]


    print(bcolors.HEADER, "Preparing Dataloaders Datasets....", bcolors.ENDC)
    #questions
    train_set_q    = QuestDataset(inputs=inputs_train_q, 
                                  usefeatures=torch.FloatTensor(xtrain_usefeatures[train_index,:]), 
                                  question_data = torch.LongTensor(train_question_tokenized[train_index,:]),
                                  answer_data = torch.LongTensor(train_answer_tokenized[train_index,:]),
                                  title_data = torch.LongTensor(train_title_tokenized[train_index,:]),
                                  lengths=lengths_train_q, labels=outputs_train_q)
    train_loader_q = DataLoader(train_set_q, batch_size=BATCH_SIZE, shuffle=True)
    #answers
    train_set_a    = QuestDataset(inputs=inputs_train_a,
                                  usefeatures=torch.FloatTensor(xtrain_usefeatures[train_index,:]), 
                                  question_data = torch.LongTensor(train_question_tokenized[train_index,:]),
                                  answer_data = torch.LongTensor(train_answer_tokenized[train_index,:]),
                                  title_data = torch.LongTensor(train_title_tokenized[train_index,:]),
                                  lengths=lengths_train_a, labels=outputs_train_a)
    train_loader_a = DataLoader(train_set_a, batch_size=BATCH_SIZE, shuffle=True)
    #questions
    valid_set_q    = QuestDataset(inputs=inputs_valid_q,
                                  usefeatures=torch.FloatTensor(xtrain_usefeatures[val_index,:]), 
                                  question_data = torch.LongTensor(train_question_tokenized[val_index,:]),
                                  answer_data = torch.LongTensor(train_answer_tokenized[val_index,:]),
                                  title_data = torch.LongTensor(train_title_tokenized[val_index,:]),
                                  lengths=lengths_valid_q, labels=outputs_valid_q)
    valid_loader_q = DataLoader(valid_set_q, batch_size=BATCH_SIZE, shuffle=False, drop_last=False)
    #answers
    valid_set_a    = QuestDataset(inputs=inputs_valid_a, 
                                  usefeatures=torch.FloatTensor(xtrain_usefeatures[val_index,:]),
                                  question_data = torch.LongTensor(train_question_tokenized[val_index,:]),
                                  answer_data = torch.LongTensor(train_answer_tokenized[val_index,:]),
                                  title_data = torch.LongTensor(train_title_tokenized[val_index,:]),
                                  lengths=lengths_valid_a, labels=outputs_valid_a)
    valid_loader_a = DataLoader(valid_set_a, batch_size=BATCH_SIZE, shuffle=False, drop_last=False)

    i = 0
    best_avg_loss_q,best_avg_loss_a    = 100.0 , 100.0
    best_score_q, best_score_a      = -1. , -1.
    best_param_loss_q, best_param_loss_a = None,None
    best_param_score_q,best_param_score_a = None,None

    #model for questions
    model_q = BertForSequenceClassification_v2Question.from_pretrained('bert-base-uncased', config=bert_config_q, embedding_matrix = embedding_matrix);
    model_q.zero_grad();
    model_q.to(device);
    torch.cuda.empty_cache()
    model_q.train();

    #model for answers
    model_a = BertForSequenceClassification_v2Answer.from_pretrained('bert-base-uncased', config=bert_config_a, embedding_matrix = embedding_matrix);
    model_a.zero_grad();
    model_a.to(device);
    torch.cuda.empty_cache()
    model_a.train();

    #questions
    optimizer_q = torch.optim.AdamW(model_q.parameters(), lr=4e-5, eps=2e-5)
    criterion = nn.BCEWithLogitsLoss( reduction='none')
    scheduler_q = get_cosine_schedule_with_warmup(optimizer_q, num_warmup_steps=0.01, num_training_steps= epochs*len(train_loader_q)//ACCUM_STEPS)

    #answers
    optimizer_a = torch.optim.AdamW(model_a.parameters(), lr=4e-5, eps=2e-5)
    criterion = nn.BCEWithLogitsLoss( reduction='none')
    scheduler_a = get_cosine_schedule_with_warmup(optimizer_a, num_warmup_steps=0.01, num_training_steps= epochs*len(train_loader_a)//ACCUM_STEPS)

    print("Training....")

    for epoch in tqdm(range(epochs),position=0, leave=True ):        
        torch.cuda.empty_cache()

        start_time_q   = time.time()
        avg_loss_q    = train_model_questions(train_loader_q, optimizer_q, criterion, scheduler_q)
        avg_val_loss_q, score_q, preds_q = val_model_questions(valid_loader_q, val_shape=val_df.shape[0])
        elapsed_time_q = time.time() - start_time_q


        start_time_a   = time.time()
        avg_loss_a    = train_model_answers(train_loader_a, optimizer_a, criterion, scheduler_a)
        avg_val_loss_a,  score_a, preds_a = val_model_answers(valid_loader_a, val_shape=val_df.shape[0])
        elapsed_time_a = time.time() - start_time_q

        p = np.hstack((preds_q,  preds_a)) #np.hstack((preds_q,  preds_a[:,21:30]))
        o = np.hstack((outputs_valid_q[:,0:21].numpy(), outputs_valid_a[:,0:9].numpy()))   #outputs_valid_a[:,0:30].numpy()
        score = np.mean([spearmanr(o[:, i], p[:,i]).correlation for i in range(30)])

        print(bcolors.WARNING, 'Epoch {}/{} \t Train loss Q={:.4f} \t Val loss Q={:.4f} \t  Score Q={:.5f} \t Train loss A={:.4f} \t Val loss A={:.4f} \t Score A={:.5f}  \t score={:.6f} \t time={:.2f}s'.format(
            epoch + 1, epochs, avg_loss_q, avg_val_loss_q,  score_q, avg_loss_a, avg_val_loss_a, score_a,  score, elapsed_time_a),
        bcolors.ENDC
        )

        logging.warning('Epoch {}/{} \t Train loss Q={:.4f} \t Val loss Q={:.4f} \t  Score Q={:.5f} \t Train loss A={:.4f} \t Val loss A={:.4f} \t Score A={:.5f}  \t score={:.6f} \t time={:.2f}s'.format(
            epoch + 1, epochs, avg_loss_q, avg_val_loss_q,  score_q, avg_loss_a, avg_val_loss_a, score_a,  score, elapsed_time_a))
        
        
        all_rhos.append(score)

        if best_score_q < score_q:
            best_score_q = score_q
            best_e_q = epoch
            best_param_score_q = model_q.state_dict()
        else:
            i += 1
            
        if best_score_a < score_a:
            best_score_a = score_a
            best_e_a = epoch
            best_param_score_a = model_a.state_dict()
        else:
            i += 1

    torch.save(model_q.state_dict(),path_join(code_dir,'saved_models','Qmodel_save_fold_{}.pt'.format(fold)))
    torch.save(model_a.state_dict(),path_join(code_dir,'saved_models','Amodel_save_fold_{}.pt'.format(fold)))
    # torch.save(best_param_score_q,path_join('/home','jupyter', 'submissions','bert_use_crawl_seed_1977','saved_models_1977','Q_model_save_fold_{}.pt'.format(fold)))
    # torch.save(best_param_score_a,path_join('/home','jupyter', 'submissions','bert_use_crawl_seed_1977','saved_models_1977','A_model_save_fold_{}.pt'.format(fold)))
    best_rhos.append((best_score_q*21 + best_score_a*9)/30.0)
    
logging.warning(np.mean(best_rhos))
print("Best Rhos:",np.mean(best_rhos))


24it [00:00, 238.64it/s]

 Current Fold: 3 
Train and Valid Shapes are (4868, 43) (1211, 43)
 Preparing train datasets.... 


4868it [00:16, 288.09it/s]
4868it [00:30, 161.20it/s]
24it [00:00, 236.10it/s]

 Preparing Valid datasets.... 


1211it [00:04, 295.11it/s]
1211it [00:07, 162.00it/s]


...

 val_spearman-rho: 0.35422                                                                                                    
0 SpearmanrResult(correlation=0.22356628161672681, pvalue=2.131842332937802e-15)
1 SpearmanrResult(correlation=0.41487980269464736, pvalue=2.333109005036306e-52)
2 SpearmanrResult(correlation=0.1496727885909048, pvalue=1.3375672128367776e-07)
3 SpearmanrResult(correlation=0.18202227610358737, pvalue=1.267055173899519e-10)
4 SpearmanrResult(correlation=0.30807308653249543, pvalue=1.8807270243976339e-28)
5 SpearmanrResult(correlation=0.7498184877907815, pvalue=1.615940058072511e-222)
6 SpearmanrResult(correlation=0.2979730593958899, pvalue=1.2186275457221916e-26)
7 SpearmanrResult(correlation=0.6580308794397478, pvalue=1.7233038294805462e-153)
8 SpearmanrResult(correlation=0.20395370842180713, pvalue=5.134236739028933e-13)
 Epoch 3/4 	 Train loss Q=3.0194 	 Val loss Q=3.2475 	  Score Q=0.43877 	 Train loss A=2.5355 	 Val loss A=2.9105 	 Score A=0.35422  	 score=0

607it [05:00,  2.64it/s]
154it [00:23,  6.99it/s]
0it [00:00, ?it/s]

 val_spearman-rho: 0.43703                                                                                                    
0 SpearmanrResult(correlation=0.3750198924990147, pvalue=2.3153923763803446e-42)
1 SpearmanrResult(correlation=0.6464447833798119, pvalue=1.8152833335796253e-146)
2 SpearmanrResult(correlation=0.43543580629077633, pvalue=4.5379009005530036e-58)
3 SpearmanrResult(correlation=0.27417910711530913, pvalue=1.1906493976408187e-22)
4 SpearmanrResult(correlation=0.41150163185617883, pvalue=1.8583491445474155e-51)
5 SpearmanrResult(correlation=0.40262628521005983, pvalue=3.87281488001904e-49)
6 SpearmanrResult(correlation=0.35546085109592274, pvalue=6.088755383813907e-38)
7 SpearmanrResult(correlation=0.5222664888611682, pvalue=5.101870880810066e-87)
8 SpearmanrResult(correlation=0.6241677642875765, pvalue=8.611858223058136e-134)
9 SpearmanrResult(correlation=0.1261107206749071, pvalue=9.160389402835418e-06)
10 SpearmanrResult(correlation=0.510173490424803, pvalue=1.807

607it [05:00,  2.65it/s]
154it [00:23,  6.97it/s]
100%|██████████| 4/4 [43:22<00:00, 650.53s/it]


 val_spearman-rho: 0.35735                                                                                                    
0 SpearmanrResult(correlation=0.23347269115640948, pvalue=1.0879192306793804e-16)
1 SpearmanrResult(correlation=0.41536875205410057, pvalue=1.7244104283921466e-52)
2 SpearmanrResult(correlation=0.15843051341282688, pvalue=2.323073817158609e-08)
3 SpearmanrResult(correlation=0.19351444652310365, pvalue=7.662975585038703e-12)
4 SpearmanrResult(correlation=0.31496091676722787, pvalue=9.94393242935773e-30)
5 SpearmanrResult(correlation=0.7477285865163545, pvalue=1.2859885936835154e-220)
6 SpearmanrResult(correlation=0.29169658342620214, pvalue=1.4990991156791277e-25)
7 SpearmanrResult(correlation=0.6505520963283843, pvalue=6.377633683389144e-149)
8 SpearmanrResult(correlation=0.21041974993154985, pvalue=8.930406383704154e-14)
 Epoch 4/4 	 Train loss Q=2.9199 	 Val loss Q=3.2614 	  Score Q=0.43703 	 Train loss A=2.4033 	 Val loss A=2.9627 	 Score A=0.35735  	 score=

In [40]:
#The Rho's are not exactly right because I reran from fold>2 (check the log)
np.mean(all_rhos) #Best Rhos: 0.4161831015275145

0.40814141312915486

In [45]:
#THIS IS THE BEST RHO
((0.43644 *21+ 0.36162*9)/30  + (0.43887*21 + 0.37817*9)/30  + (0.43403*21 + 0.39442*9)/30  + (0.43123*21 + 0.36439*9)/30 + (0.43901*21 + 0.36131*9)/30 ) /5 

0.4167358

In [41]:
all_rhos

[0.39624068987107164,
 0.4091913761787977,
 0.4079087894316589,
 0.40814334031265537,
 0.4014168467334519,
 0.41569720881375893,
 0.4134059818059639,
 0.4131270718858805]

0.4167358

In [0]:
#Log for first 3 folds

# WARNING:root:Epoch 1/4 	 Train loss Q=3.4740 	 Val loss Q=3.2735 	  Score Q=0.42140 	 Train loss A=2.9563 	 Val loss A=2.8575 	 Score A=0.34180  	 score=0.397518 	 time=632.98s
# WARNING:root:Epoch 2/4 	 Train loss Q=3.1800 	 Val loss Q=3.2323 	  Score Q=0.43644 	 Train loss A=2.7395 	 Val loss A=2.8126 	 Score A=0.36162  	 score=0.413996 	 time=633.03s
# WARNING:root:Epoch 3/4 	 Train loss Q=3.0293 	 Val loss Q=3.2350 	  Score Q=0.43579 	 Train loss A=2.5497 	 Val loss A=2.9032 	 Score A=0.35433  	 score=0.411354 	 time=632.00s
# WARNING:root:Epoch 4/4 	 Train loss Q=2.9316 	 Val loss Q=3.2509 	  Score Q=0.43440 	 Train loss A=2.4209 	 Val loss A=2.9351 	 Score A=0.34896  	 score=0.408764 	 time=632.60s

# WARNING:root:Epoch 1/4 	 Train loss Q=3.4762 	 Val loss Q=3.3138 	  Score Q=0.41558 	 Train loss A=2.9663 	 Val loss A=2.8026 	 Score A=0.36065  	 score=0.399100 	 time=633.43s
# WARNING:root:Epoch 2/4 	 Train loss Q=3.1721 	 Val loss Q=3.2465 	  Score Q=0.43665 	 Train loss A=2.7491 	 Val loss A=2.7607 	 Score A=0.37817  	 score=0.419110 	 time=633.50s
# WARNING:root:Epoch 3/4 	 Train loss Q=3.0190 	 Val loss Q=3.2451 	  Score Q=0.43887 	 Train loss A=2.5622 	 Val loss A=2.8191 	 Score A=0.37341  	 score=0.419230 	 time=633.16s
# WARNING:root:Epoch 4/4 	 Train loss Q=2.9192 	 Val loss Q=3.2632 	  Score Q=0.43659 	 Train loss A=2.4302 	 Val loss A=2.8614 	 Score A=0.36607  	 score=0.415435 	 time=633.45s

# WARNING:root:Epoch 1/4 	 Train loss Q=3.4953 	 Val loss Q=3.2774 	  Score Q=0.41193 	 Train loss A=2.9427 	 Val loss A=2.8652 	 Score A=0.38094  	 score=0.402633 	 time=632.93s
# WARNING:root:Epoch 2/4 	 Train loss Q=3.1822 	 Val loss Q=3.2347 	  Score Q=0.42978 	 Train loss A=2.7365 	 Val loss A=2.8441 	 Score A=0.39442  	 score=0.419169 	 time=634.40s
# WARNING:root:Epoch 3/4 	 Train loss Q=3.0290 	 Val loss Q=3.2249 	  Score Q=0.43403 	 Train loss A=2.5501 	 Val loss A=2.8867 	 Score A=0.38308  	 score=0.418748 	 time=632.01s
# WARNING:root:Epoch 4/4 	 Train loss Q=2.9303 	 Val loss Q=3.2448 	  Score Q=0.43117 	 Train loss A=2.4134 	 Val loss A=2.9508 	 Score A=0.37948  	 score=0.415664 	 time=632.38s

